In [118]:
lines="""*+Specification+*
*User Control Panel*
*FRS2881.0* Accessing
The user will click the "Account" button in the Shell, and the "Manage" button beneath it.
# If the user has "Account Management" permissions, they will be brought to the Configure Users tab of the User Control Panel, and the first name in the User List will be selected.
# If the user does not have "Account Management" permissions, the "Edit User" VI will open, for their own user account.

*FRS2881.1* Configure Users tab
# Mockup
!Configure_Users.png!
# Numbered mockup
!Configure_Users_Numbered.png!

*FRS2881.11* Configure Users tab Appearance
# Item 1 - Configure Users tab
## This will be a tab
## It will be labeled Configure Users
## It will be the leftmost tab in the UI
# Item 2 - Configure Groups tab.  See its FRS section for its full spec.
# Item 3 - User List
## It will be labeled User List
## It will have 2 columns
### The first column will be labeled User Name
### The second column will be labeled Group
# Item 4 - Edit User Boolean button
# Item 5 - Add User Boolean button
# Item 6 - Delete User Boolean button

*FRS2881.12* Configure Users tab Functionality
# Item 1
## This tab will be selected when the user first navigates to this UI
## When selected, the tab will be larger in size than the other tab
## When not selected, the tab will be smaller in size than the other tab
## When clicked while not selected, it will show the Configure Users tab
# Item 2 See its FRS section for its full spec.
# Item 3
## This will be populated with the user accounts in the PBSUsers.conf file
## Selecting items
### When an item is selected, it will be highlighted blue and its text will be white
### 1 item is to be selected at all times
### No more than 1 item is to be selected at any time
### When the UI is navigated to, the first item will be selected
# Item 4 - Edit User button
## When clicked, this will open the Edit User dialog, and pass it the selected User Name
# Item 5 - Add User button
## When clicked, this will open the Edit User dialog, and pass it a "New User".  See below for further specifications for the Edit User dialog for this condition.
# Item 6 - Delete User button
## When clicked, the selected user will be deleted, and the first user in the list will be selected.  A User Event will be recorded with the message "Deleted User X" with X being the account name of the deleted user and the user who initiated this event.
## This button will be grayed out under the following conditions:
### If there is only 1 user in the User List
### If the selected user in the User List is the current user (i.e. a user won't be able to delete their own account)
# When a user is added, edited or deleted, or a group name is edited, refresh the user list to reflect the changes


*FRS2881.2* Edit User menu
# Mockup
!Edit_User.png!
# Numbered mockup
!Edit_User_Numbered.png!

*FRS2881.21* Edit User menu Appearance
# This UI will be labeled Edit User
# Item 1 - a string input labeled Username
# Item 2 and 3 - a string input labeled Enter Password; Use password display for both
# Item 4 - a string input labeled Email Address
# Item 5 - a listbox labeled Group
# Item 6 - a string indicator labeled Days Until Password Expires
# Item 7 - a string indicator labeled Alert
# Item 8 - a Boolean button labeled Cancel
# Item 9 - a Boolean button labeled Save and Close

*FRS2881.22* Edit User menu Functionality
# This VI will be accessed via the following methods:
## A user without Account Management permissions clicking "Account" > "Manage" in the Shell.  This will open the dialog, after passing it the user's account name.
## A user with Account Management permissions selecting an account in the User List in the Configure Users tab of the User Control Panel, and clicking "Edit User".  This will open the dialog, after passing it the selected user's account name.
## A user with Account Management permissions clicking "Add User" in the Configure Users tab of the User Control Panel.  This will open the dialog, after passing it "New User"
# When the VI is opened and a user's account name was passed to it:
### Item 1 will contain the user's Username
### Items 2 and 3 will be blank
### Item 6 will show the correct calculation
### The user's Group, as saved in the PBSUsers.conf file, will be selected in item 7
### Item 8 will show the user's email address, as saved in the PBSUsers.conf file
### Item 9 will show any errors it needs to (see below)
## When the VI is opened and "New User" was passed to it:
### Items 2, 3, 4 will be empty
### Item 5 will have the first Group in the list selected
### Item 6 will show the calculation for the first group
### Item 7 will show any errors it needs to (see below)
# When this menu is active, all Shell buttons used for navigation, logging out etc. will be grayed out.  This will force the user to either click "Save and Close" or "Cancel" to get out of the menu.
# Item 1
## If the user has Account Management permissions, they will be able to modify this field.  Otherwise, it will be read-only.
## This field will be used to modify the user name of the selected account
## Changing the contents of this field will trigger item 7 to update
# Items 2 and 3
## These will be used to modify the password of the user account
## Changing the contents of these fields will trigger item 7 to update
# Item 4
## This will display the saved email address for the selected user
## It will also be used to modify the saved email address for the selected user
## Changing the contents of this field will trigger item 7 to update
# Item 5
## This will be populated with the groups from the PBSUsers.conf file
## Selecting items
### When an item is selected and the user has Account Management permissions, it will be highlighted blue and its text will be white.
### 1 item is to be selected at all times
### No more than 1 item is to be selected at any time
### Only users with Account Management permissions can modify the selected group - for all other accounts, this will be read-only.
### Selecting a different group will trigger item 7 to update
# Item 6
## If the user's assigned group's Password Expiration Period is 0, this will show "Never"
## If it's a new user, and their assigned group's Password Expiration Period is not 0, this will show 0
## Otherwise, the VI will calculate the number of days until the user's password expires, based on when the user last saved their password, the user's assigned group's Password Expiration Period, and today's date.  If it has already expired, show 0
# Item 7
## If there is no alert, it will not be visible
## Any/all of the following alerts can be shown at the same time.  If displaying more than 1 alert, they do not have to be displayed in the following order.
### You must make a change in order to Save
If the account exists in the PBSUsers.conf file, items 2 and 3 are empty, and the contents of items 1, 4 and 5 match what's saved for that user
### User Names must be unique
If the contents of item 1 match another account's User Name
### User Name cannot be blank
If item 1 is empty
### The entered Passwords do not match
If the contents of items 2 and 3 do not match
### Passwords must be at least 5 characters long
If item 2 is not empty, and its contents are less than 5 characters long
or
If items 2 and 3 are empty, and the dialog was opened with the "New User" flag
### This user's Group has Hello Access permissions, so the entered Password must only contain numbers
If the selected group in item 5 has Hello Access permissions, and item 2 is not empty, and item 2 contains non-numeric characters
or
If the selected group in item 5 has Hello Access permissions, and items 2 and 3 are empty, and the saved password for the account contains non-numeric characters
### The entered email address is not valid
If item 4 is not empty, and does not contain a string in the format of X@Y.Z where X, Y and Z are each at least 1 character, none of which are @
# Item 8
## When clicked, the Edit User menu will close and the user will be returned to the previous VI they were accessing.  No user events will be created, and no changes will be saved to the PBSUsers.conf file.
# Item 9
## This button will be grayed out if there are any Alerts.  If clicked when it is grayed out, nothing will happen.
## When this button is clicked and it isn't grayed out:
### If it's a new user:
#### The new user account will be created in the PBSUsers.conf file, using information from items 1, 2, 4 and 5  (item 2 will already have been confirmed to match item 3).  The user's "Number of Failed Login Attempts", stored in the PBSUsers.conf file, will be set to 0, and the user's "Date Password was Saved", stored in the PBSUsers.conf file, will be set to the current date.
#### A user event will be created "Created new user <User Name> in group <Group> with email address <Email Address>" where the bracketed items are replaced with the contents of items 1, 5 and 4, respectively.
#### The VI will close and the user will be returned to the previous VI they had been accessing.
### If it's an existing user:
#### Write the appropriate items to the PBSUsers.conf file.  If item 2 isn't empty, write the new encrypted password, set the "Number of Failed Login Attempts" to 0, and the "Date Password was Saved" to today's date.
#### Create a user event where the first line is "User <Item 1>" where instead of <Item 1> have the contents of item 1, the second line is "Group <Item 5>", and the third line is "Email Address <Item 4>".  It can also have any of the following lines as well.  If they are displayed, they should be in the following order.
##### If item 1 doesn't match the saved User Name for the user in the PBSUsers.conf file, "User Name was changed from <Original User Name> to <New User Name>"
##### If item 2 isn't empty, "Password was changed"
#### The VI will close and the user will be returned to the previous VI they had been accessing.


*FRS2881.3* Configure Groups tab
# Mockup
!Configure_Groups.png!
# Numbered mockup
!Configure_Groups_Numbered.png!

*FRS2881.31* Configure Groups tab Appearance
# Item 1 - Configure Users tab
## This tab will be smaller than the Configure Groups tab (item 2) when the Configure Groups tab is selected.
# Item 2 - Configure Groups tab
## This tab will be bigger than item 1 when it's selected
# Item 3 - Permissions list
## The main text of this list is to be displayed at 90°
## The sections in the list are to be displayed in bold, horizontally, as in the mockup.
## Lines will be used, as in the mockup, to divide the sections.
## This list is to be divided into the following sections, and displayed in the following order.  
### *Desktop*
#### Alarms Editor
#### Logger Editor
#### Settings Editor
#### Recipe Editor
#### Advanced Calibration
#### DB Management
#### Account Management
### *Hello Interface*
#### Hello Access
#### Remote Access
#### Door
#### Light
#### Batch
#### Recipe Start
#### Recipe End
#### Pumps
#### Acknowledge Alarms
#### Sensors
#### Hello Settings
### *Common*
#### Reports
#### Controls
#### System Management
# Item 4 - a listbox labeled Group Name
# Item 5 - a listbox labeled Password Expiration Period (days)
# Item 6 - a 2-D array of Booleans with no labels
## True will be bright green, and False will be dark green
## An empty row, for a nonexistent group, will be gray
# Item 7 - a scrollbar for scrolling up and down to see additional groups; it will not be labeled
# Item 8 - a Boolean button labeled Add Group
# Item 9 - a Boolean button labeled Delete Group
# Item 10 - a Boolean button labeled Save
# Item 11 - a Boolean button labeled Reset

*FRS2881.32* Configure Groups tab Functionality
# When a user makes any change in this menu, the Shell navigation buttons get grayed out so a user has to click the Save or Reset buttons.
# Item 1
## If unsaved changes are made in the Configure Groups tab, this tab will be grayed out.  It will no longer be grayed out after clicking "Save" or "Reset".
# Item 2
## No functionality beyond what's already been covered here
# Item 3
## This is static text and thus has no function beyond what's described above in Appearance
# Item 4
## Only one item can be selected at all times
## The corresponding item in Item 5 will also be selected
## When an item is selected, the background will be blue and the text will be white
## When the Configure Groups tab is accessed, the first item in the list will be selected
## Empty rows cannot be selected
## A blank row will have gray controls for item 6, and items 4 and 5 will be blank
## Double clicking allows a user to modify the contents of item 4 for a particular group
## If the user clears the item and attempts to navigate away, it will be repopulated with the old value
## If the user enters the same name for item 4 for a group that already exists, it will be repopulated with the old value
# Item 5
## Only one item can be selected at all times
## The corresponding item in Item 4 will also be selected
## When an item is selected, the background will be blue and the text will be white
## When the Configure Groups tab is accessed, the first item in the list will be selected
## Empty rows cannot be selected
## A blank row will have gray controls for item 6, and items 4 and 5 will be blank
## Double clicking allows a user to modify the contents of item 5 for a particular group
## If the user clears the item and attempts to navigate away, it will be repopulated with 0
## Only integers greater than or equal to 0 will be entered
# Item 6
## The users can click here and assign permissions to groups in this table
# Item 7
## Allows the user to scroll up and down to see groups beyond the 8 visible ones
## If there are less than 9 groups, the scrollbar shall not be usable.
## It will be applicable for Item 4, 5, and 6.
# Item 8
## If there is already a blank group and the user attempts to add another blank group, display the error message "Please edit the blank group first. Once the blank group is edited, you can add another group." 
## Clicking this will create a new group in the table, and select it.  The new group will be empty for item 4, be 0 for item 5, and item 6 will be all 'FALSE'
# Item 9
## Clicking this button will check the number of users assigned to the group, and the number of groups displayed.
### If it is the last group in the list, an alert VI (not a true popup) will be displayed with the message "A group cannot be deleted if it is the only group remaining in the list" and 1 button, "OK".  When the "OK" button is clicked, the alert VI will close.
### If there are users assigned to the group, an alert VI (not a true popup) will be displayed with the message "A group cannot be deleted if there are users assigned to it" and 1 button, "OK".  When the "OK" button is clicked, the alert VI will close.
### Otherwise, the selected group will be deleted from the table, and the top group in the list will be selected.
# Item 10
## Clicking this button will check that no groups have empty names (checking for duplicate names is done elsewhere).
### If any group names are blank, an alert VI (not a true popup) will be displayed with the message "Groups with blank names cannot be saved" and 1 button, "OK".  When the "OK" button is clicked, the alert VI will close.
### Otherwise,  it will write the table to the PBSUsers.conf file, and a user event will be created, logging the contents of the table.  The Shell navigation buttons that the user has permissions for will then no longer be grayed out.  Also Item 1 will no longer be grayed out.
# Item 11
## Clicking this button will cancel all changes the user made, refresh the menu to show the current contents of the PBSUsers.conf file, and the Shell navigation buttons that the user has permissions for will then no longer be grayed out.  Also Item 1 will no longer be grayed out.

*PBSUsers.conf file*
*FRS2881.4*
# PBSUsers.conf file will keep track of the following data:
## Per User
### User Name
### Group
### Password
### Email Address
### Date Password was Saved
### Number of Failed Login Attempts
## Per Group
### Group Name
### Password Expiration Period (Days)
## 2-D Permission Table
### Each row corresponds to each group"""

In [142]:
import re
frs_match = re.compile(r"^(?:([\+\*]*) )?([\+\* ]*)(FRS\d+)\.?([\d\.]+)?([\+\*\:]*)?\s*(.+)?$").match
subitem_match = re.compile(r"^(?:([#]+) )").match
magic_match = re.compile(r"^@lm\:\<(.+)\>").match

class NoMatch(Exception):
    pass

class NumState():
    VALID_INDENTS = "*+"
    LINE_FRS = 0
    LINE_SUBITEM = 1
    LINE_OTHER = 2
    def __init__(self, depth=None, startval=1, indent_char="*"):
        self._startval = startval
        self.state = self._newstate()
        self.ilvl = 0
        self.nlvl = 0 
        self.clvl = 0
        self.frs = ""
        self.nums = ""
        self.fmt = ""
        self.depth = depth
        if indent_char not in self.VALID_INDENTS:
            raise ValueError("Invalid Indent Character " + indent_char)
        self.indent_char = indent_char
        self.line = ""
        
    def _newstate(self):
        return [self._startval]
        
    def _grow(self, n):
        while len(self.state) < n:
            self.state.append(self._startval)
    
    def set_state(self, lvl, val):
        self._grow(lvl)
        self.state[lvl] = val
    
    def set_state_from_nums(self, nums):
        if not nums:
            lnums = []
        else:
            lnums = nums.split(".")
        self.state = [self._startval] * len(lnums)
        for i, n in enumerate(lnums):
            self.state[i] = int(n)
        return len(lnums)
    
    def set_str(self, s):
        m = frs_match(s)
        if not m:
            raise NoMatch(s)
        return self.set_match(m)

    def set_match(self, m):
        indent, pre_fmt, frs, nums, post_fmt, bob = m.groups()
        self.ilvl = len(indent or "")
        self.fmt = pre_fmt
        self.frs = frs
        self.nlvl = self.set_state_from_nums(nums)
        self.clvl = 0
        
    def frs_string(self):
        return self.format_lvl(self.clvl)
    
    def indent(self, n=1):
        maxn = self.clvl + self.nlvl + n
        minn = self.clvl + self.nlvl
        self._grow(maxn)
        for i in range(minn, maxn):
            self.state[i] = 1
        self.clvl += n
    
    def dedent(self, n=1):
        self.clvl -= n
        self.next()
        
    def next(self):
        self.state[self.clvl+self.nlvl-1] += 1
        
    def feed(self, line):
        self.line = line
        m = frs_match(line)
        if m:
            self.set_match(m)
            return self.LINE_FRS
        m = subitem_match(line)
        if m:
            self.set_indent(len(m.group(1)))
            return self.LINE_SUBITEM
        return self.LINE_OTHER
    
    def set_indent(self, n):
        diff = n - (self.ilvl + self.clvl)
        if diff == 0:
            self.next()
        elif diff > 0:
            self.indent(diff)
        else:
            self.dedent(-diff)
            
    def curr_state(self, lvl):
        return self.state[:self.nlvl + lvl]
        
    def format_lvl(self, lvl):
        idl = lvl + self.ilvl
        if idl < 0: idl = 0
        ind = self.indent_char * idl
        if self.depth is not None and idl > self.depth:
            return ind
        space = " " if ind else ""
        frs = self.frs
        fmt = "*"
        nums = ".".join(str(n) for n in self.curr_state(lvl))
        dot = "." if nums else ""
        post = ":*"
        return "".join((ind, space, fmt, frs, dot, nums, post))
    
    def parse_text(self, text):
        lines = text.splitlines()
        for line in lines:
            typ = self.feed(line)
            if typ == self.LINE_FRS:
                v = line
            elif typ == self.LINE_SUBITEM:
                fmt = self.frs_string()
                v = " ".join((fmt, line.split(" ", 1)[1]))
            elif typ == self.LINE_OTHER:
                v = line
            else:
                raise ValueError(typ)
            yield v

In [143]:
def test_match(s):
    return frs_match(s).groups()

def assert_match(s, exp):
    m = test_match(s)
    assert m == exp, (m, exp)
    
assert_match("*FRS1234*", (None, "*", "FRS1234", None, "*", None))
assert_match("* *FRS1234*", ("*", "*", "FRS1234", None, "*", None))
assert_match("*FRS1234.5.6*", (None, "*", "FRS1234", "5.6", "*", None))
assert_match("*+FRS1234.5.6+*", (None, "*+", "FRS1234", "5.6", "+*", None))
assert_match("** *+FRS1234.5.6.7.8+*", ("**", "*+", "FRS1234", "5.6.7.8", "+*", None))
assert_match("* *FRS1234:*", ("*", "*", "FRS1234", None, ":*", None))
assert_match("* *FRS1234*:", ("*", "*", "FRS1234", None, "*:", None))
assert_match("* *FRS1234*: Bob", ("*", "*", "FRS1234", None, "*:", "Bob"))

def assert_ab(a, b):
    try:
        assert a == b, "%s != %s" % (a, b)
    except AssertionError as e:
        print(e.args)
        raise

def test_state(s):
    state = NumState()
    state.set_str(s)
    return state
    
def assert_state(s, statevals, ilvl, nlvl):
    state = test_state(s)
    assert_ab(state.ilvl, ilvl)
    assert_ab(state.nlvl, nlvl)
    assert_ab(state.state, statevals)

def test_ilvl(s, ilvl):
    state = NumState()
    state.set_str(s)
    return state
    
def assert_ilvl(s, ilvl):
    state = test_ilvl(s, ilvl)
    assert_ab(state.ilvl, ilvl)
    
def test_fmt(s):
    state = NumState()
    state.set_str(s)
    return state
    
def assert_fmt1(s, id, fmt):
    state = test_fmt(s)
    state.indent(id)
    assert_ab(state.frs_string(), fmt)
    
def assert_fmt2(s, id, fmt):
    state = test_fmt(s)
    for _ in range(id):
        state.indent(1)
    assert_ab(state.frs_string(), fmt)
    
def assert_indent(s, id, fmt):
    assert_fmt1(s, id, fmt)
    assert_fmt2(s, id, fmt)

assert_ilvl("*FRS1234*", 0)
assert_ilvl("*FRS1234.5.6*", 0)
assert_ilvl("** *FRS1234.5.6*", 2)

assert_state("*FRS1234*", [], 0, 0)
assert_state("*FRS1234.5.6*", [5, 6], 0, 2)
assert_state("** *FRS1234.5.6*", [5, 6], 2, 2)
assert_state("** *FRS1234*", [], 2, 0)

def assert_state2(s, ilvl, nlvl, clvl):
    state = test_state(s)
    st = state.state.copy()
    state.indent()
    st.append(1)
    assert_ab(state.state, st)
    assert_ab(state.ilvl, ilvl)
    assert_ab(state.nlvl, nlvl)
    assert_ab(state.clvl, clvl)

assert_state2("*FRS1234*", 0, 0, 1)
assert_state2("* *FRS1234*", 1, 0, 1)
assert_indent("*FRS1234*", 1, "* *FRS1234.1:*")
assert_indent("*FRS1234*", 1, "* *FRS1234.1:*")
assert_indent("* *FRS1234*", 1, "** *FRS1234.1:*")
assert_indent("*FRS1234.1", 1, "* *FRS1234.1.1:*")
assert_indent("*FRS1234.1", 2, "** *FRS1234.1.1.1:*")

def assert_next(s, exp):
    st = NumState()
    st.set_str(s)
    st.next()
    assert_ab(st.frs_string(), exp)

def assert_dedent(s, id, nxt, dd, fmt, typ=1):
    st = NumState()
    st.set_str(s)
    st.indent(id)
    for _ in range(nxt):
        st.next()
    if typ == 1:
        st.dedent(dd)
    else:
        for _ in range(dd):
            st.dedent(1)
    assert_ab(st.frs_string(), fmt)

assert_next("*FRS1234.1*", "*FRS1234.2:*")
assert_next("* *FRS1234.1*", "* *FRS1234.2:*")
assert_next("*FRS1234.1.2*", "*FRS1234.1.3:*")
assert_next("* *FRS1234.1.2*", "* *FRS1234.1.3:*")
assert_dedent("*FRS1234.1*", 1, 2, 1, "*FRS1234.2:*")
assert_dedent("*FRS1234.1*", 1, 1, 1, "*FRS1234.2:*")
assert_dedent("*FRS1234.1*", 2, 1, 1, "* *FRS1234.1.2:*")
assert_dedent("*FRS1234.1*", 2, 1, 1, "* *FRS1234.1.2:*")
assert_dedent("*FRS1234.1*", 2, 1, 2, "*FRS1234.2:*")
assert_dedent("*FRS1234.1*", 3, 1, 2, "* *FRS1234.1.2:*")
assert_dedent("* *FRS1234.1*", 1, 2, 1, "* *FRS1234.2:*")
assert_dedent("* *FRS1234.1*", 1, 1, 1, "* *FRS1234.2:*")
assert_dedent("* *FRS1234.1*", 2, 1, 1, "** *FRS1234.1.2:*")
assert_dedent("* *FRS1234.1*", 2, 1, 1, "** *FRS1234.1.2:*")
assert_dedent("* *FRS1234.1*", 2, 1, 2, "* *FRS1234.2:*")
assert_dedent("* *FRS1234.1*", 3, 1, 2, "** *FRS1234.1.2:*")

def assert_feed_indent(s, feed, exp):
    st = NumState()
    st.set_str(s)
    st.feed(feed)
    assert_ab(st.frs_string(), exp)
    
assert_feed_indent("*FRS1234*", "# Bob", "* *FRS1234.1:*")
assert_feed_indent("*FRS1234*", "## Bob", "** *FRS1234.1.1:*")
assert_feed_indent("*FRS1234.1*", "# Bob", "* *FRS1234.1.1:*")
assert_feed_indent("*FRS1234.1*", "## Bob", "** *FRS1234.1.1.1:*")
assert_feed_indent("* *FRS1234.1*", "# Bob", "* *FRS1234.2:*")
assert_feed_indent("* *FRS1234*", "## Bob", "** *FRS1234.1:*")
assert_feed_indent("** *FRS1234.1*", "## Bob", "** *FRS1234.2:*")
assert_feed_indent("* *FRS1234.1*", "## Bob", "** *FRS1234.1.1:*")
assert_feed_indent("** *FRS1234.1.1", "# Bob", "* *FRS1234.2:*")

def assert_double_parse(t, d=5):
    r = list(NumState(d).parse_text(t))
    r2 = list(NumState(d).parse_text("\n".join(r)))
    assert_ab(r, r2)
    
dpt = """*FRS2881.4*
# PBSUsers.conf file will keep track of the following data:
## Per User
### User Name
### Group
### Password
### Email Address
### Date Password was Saved
### Number of Failed Login Attempts
## Per Group
### Group Name
### Password Expiration Period (Days)
## 2-D Permission Table
### Each row corresponds to each group"""
assert_double_parse(dpt)

In [128]:
depth=2
parsed = NumState(depth).parse_text(lines)
l = []
for l1, l2 in zip(lines.splitlines(), parsed):
    l.append(("%-25s" % l1[:20], l2[:30]))
for i in l:
    print(*i)

*+Specification+*         *+Specification+*
*User Control Panel*      *User Control Panel*
*FRS2881.0* Accessin      *FRS2881.0* Accessing
The user will click       The user will click the "Accou
# If the user has "A      * *FRS2881.0.1:* If the user h
# If the user does n      * *FRS2881.0.2:* If the user d
                          
*FRS2881.1* Configur      *FRS2881.1* Configure Users ta
# Mockup                  * *FRS2881.1.1:* Mockup
!Configure_Users.png      !Configure_Users.png!
# Numbered mockup         * *FRS2881.1.2:* Numbered mock
!Configure_Users_Num      !Configure_Users_Numbered.png!
                          
*FRS2881.11* Configu      *FRS2881.11* Configure Users t
# Item 1 - Configure      * *FRS2881.11.1:* Item 1 - Con
## This will be a ta      ** *FRS2881.11.1.1:* This will
## It will be labele      ** *FRS2881.11.1.2:* It will b
## It will be the le      ** *FRS2881.11.1.3:* It will b
# Item 2 - Configure      * *FRS2881.11.2:* Item 2 - Con
# Item 3 - User List     

In [141]:
lines="""*+Specification+*
*User Control Panel*
*FRS2881.0* Accessing
The user will click the "Account" button in the Shell, and the "Manage" button beneath it.
# If the user has "Account Management" permissions, they will be brought to the Configure Users tab of the User Control Panel, and the first name in the User List will be selected.
# If the user does not have "Account Management" permissions, the "Edit User" VI will open, for their own user account.

*FRS2881.1* Configure Users tab
Mockup
!Configure_Users.png!
Numbered mockup
!Configure_Users_Numbered.png!

*FRS2881.1.1* Configure Users tab Appearance
* Item 1 - Configure Users tab
** This will be a tab labeled "Configure Users"
** It will be the leftmost tab in the UI
* Item 2 - Configure Groups tab.
* Item 3 - User List
** It will be labeled User List
** It will have 2 columns
*** The first column will be labeled User Name
*** The second column will be labeled Group
* Item 4 - Edit User Boolean button
* Item 5 - Add User Boolean button
* Item 6 - Delete User Boolean button

*FRS2881.1.2* Configure Users tab Functionality
# Item 1
## This tab will be selected when the user first navigates to this UI
## When selected, the tab will be larger in size than the other tab
## When not selected, the tab will be smaller in size than the other tab
## When clicked while not selected, it will show the Configure Users tab
# Item 3
## This will be populated with the user accounts in the PBSUsers.conf file
## When an item is selected, it will be highlighted blue and its text will be white
## Exactly 1 item must always be selected - never more or less. 
## When the UI is navigated to, the first item will be selected
# Item 4 - Edit User button: When clicked, this will open the Edit User dialog, and pass it the selected User Name
# Item 5 - Add User button: When clicked, this will open the Edit User dialog, and pass it a "New User".
# Item 6 - Delete User button
## When clicked, the selected user will be deleted, and the first user in the list will be selected.  A User Event will be recorded with the message "Deleted User X" with X being the account name of the deleted user and the user who initiated this event.
## The button will be grayed out if there is only 1 user in the User List
## The button will be grayed out if the selected user in the User List is the current user (i.e. a user won't be able to delete their own account)
# When a user is added, edited or deleted, or a group name is edited, the user list will refresh to reflect the changes

*FRS2881.2* Edit User menu
Mockup
!Edit_User.png!
Numbered mockup
!Edit_User_Numbered.png!

*FRS2881.2.1* Edit User menu Appearance
* This UI will be labeled Edit User
* Item 1 - a string input labeled Username
* Item 2 and 3 - a string input labeled Enter Password; Use password display for both
* Item 4 - a string input labeled Email Address
* Item 5 - a listbox labeled Group
* Item 6 - a string indicator labeled Days Until Password Expires
* Item 7 - a string indicator labeled Alert
* Item 8 - a Boolean button labeled Cancel
* Item 9 - a Boolean button labeled Save and Close

*FRS2881.2.2* Edit User menu Functionality
# This VI will be accessed via the following methods:
## A user without Account Management permissions clicking "Account" > "Manage" in the Shell.  This will open the dialog, after passing it the user's account name.
## A user with Account Management permissions selecting an account in the User List in the Configure Users tab of the User Control Panel, and clicking "Edit User".  This will open the dialog, after passing it the selected user's account name.
## A user with Account Management permissions clicking "Add User" in the Configure Users tab of the User Control Panel.  This will open the dialog, after passing it "New User"
# When the VI is opened and a user's account name was passed to it:
## Item 1 will contain the user's Username
## Items 2 and 3 will be blank
## Item 6 will show the correct calculation
## The user's Group, as saved in the PBSUsers.conf file, will be selected in item 7
## Item 8 will show the user's email address, as saved in the PBSUsers.conf file
## Item 9 will show any errors it needs to (see below)
# When the VI is opened and "New User" was passed to it:
## Items 2, 3, 4 will be empty
## Item 5 will have the first Group in the list selected
## Item 6 will show the calculation for the first group
## Item 7 will show any errors it needs to (see below)
# When this menu is active, all Shell buttons used for navigation, logging out etc. will be grayed out.  This will force the user to either click "Save and Close" or "Cancel" to get out of the menu.
# Item 1
## If the user has Account Management permissions, they will be able to modify this field.  Otherwise, it will be read-only.
## This field will be used to modify the user name of the selected account
## Changing the contents of this field will trigger item 7 to update
# Items 2 and 3 will be used to modify the password of the user account
## Changing the contents of these fields will trigger item 7 to update
# Item 4
## This will display the saved email address for the selected user
## It will also be used to modify the saved email address for the selected user
## Changing the contents of this field will trigger item 7 to update
# Item 5
## This will be populated with the groups from the PBSUsers.conf file
## When an item is selected and the user has Account Management permissions, it will be highlighted blue and its text will be white.
## Exactly 1 item must always be selected - never more or less. 
## Only users with Account Management permissions can modify the selected group - for all other accounts, this will be read-only.
## Selecting a different group will trigger item 7 to update
# Item 6
## If the user's assigned group's Password Expiration Period is 0, this will show "Never"
## If it's a new user, and their assigned group's Password Expiration Period is not 0, this will show 0
## Otherwise, the VI will calculate the number of days until the user's password expires, based on when the user last saved their password, the user's assigned group's Password Expiration Period, and today's date.  If it has already expired, show 0
# Item 7
## If there is no alert, it will not be visible
## Any/all of the following alerts can be shown at the same time. 
## If displaying more than 1 alert, they do not have to be displayed in the following order.
### "You must make a change in order to Save": If the account exists in the PBSUsers.conf file, items 2 and 3 are empty, and the contents of items 1, 4 and 5 match what's saved for that user
### "User Names must be unique": If the contents of item 1 match another account's User Name
### "User Name cannot be blank": If item 1 is empty
### "The entered Passwords do not match": If the contents of items 2 and 3 do not match
### "Passwords must be at least 5 characters long": If item 2 is not empty and its contents are less than 5 characters long, or If items 2 and 3 are empty, and the dialog was opened with the "New User" flag
### "This user's Group has Hello Access permissions, so the entered Password must only contain numbers" In the following circumstances:
**** If the selected group in item 5 has Hello Access permissions, and item 2 is not empty, and item 2 contains non-numeric characters
**** If the selected group in item 5 has Hello Access permissions, and items 2 and 3 are empty, and the saved password for the account contains non-numeric characters
### "The entered email address is not valid": If item 4 is not empty, and does not contain a string in the format of X@Y.Z where X, Y and Z are each at least 1 character, none of which are @
# When Item 8 is clicked, the Edit User menu will close and the user will be returned to the previous VI they were accessing.  No user events will be created, and no changes will be saved to the PBSUsers.conf file.
# Item 9 
## Will be grayed out if there are any Alerts.
## If clicked when it is grayed out, nothing will happen.
## When this button is clicked and it isn't grayed out, and it is a new user:
### The new user account will be created in the PBSUsers.conf file, using information from items 1, 2, 4 and 5  (item 2 will already have been confirmed to match item 3).  The user's "Number of Failed Login Attempts", stored in the PBSUsers.conf file, will be set to 0, and the user's "Date Password was Saved", stored in the PBSUsers.conf file, will be set to the current date.
### A user event will be created "Created new user <User Name> in group <Group> with email address <Email Address>" where the bracketed items are replaced with the contents of items 1, 5 and 4, respectively.
### The VI will close and the user will be returned to the previous VI they had been accessing.
## When this button is clicked and it isn't grayed out, and it is an existing user:
### Write the appropriate items to the PBSUsers.conf file.  If item 2 isn't empty, write the new encrypted password, set the "Number of Failed Login Attempts" to 0, and the "Date Password was Saved" to today's date.
### Create a user event where the first line is "User <Item 1>" where instead of <Item 1> have the contents of item 1, the second line is "Group <Item 5>", and the third line is "Email Address <Item 4>".  It can also have any of the following lines as well.  If they are displayed, they should be in the following order.
#### If item 1 doesn't match the saved User Name for the user in the PBSUsers.conf file, "User Name was changed from <Original User Name> to <New User Name>"
#### If item 2 isn't empty, "Password was changed"
### The VI will close and the user will be returned to the previous VI they had been accessing.

*FRS2881.3* Configure Groups tab
Mockup
!Configure_Groups.png!
Numbered mockup
!Configure_Groups_Numbered.png!

*FRS2881.3.1* Configure Groups tab Appearance
# Item 1 - Configure Users tab will be smaller than the Configure Groups tab (item 2) when the Configure Groups tab is selected.
# Item 2 - Configure Groups tab will be bigger than item 1 when it's selected
# Item 3 - Permissions list
## The main text of this list is to be displayed at 90°
## The sections in the list are to be displayed in bold, horizontally, as in the mockup.
## Lines will be used, as in the mockup, to divide the sections.
## This list is to be divided into the following sections, and displayed in the following order.  
### *Desktop*
**** Alarms Editor
**** Logger Editor
**** Settings Editor
**** Recipe Editor
**** Advanced Calibration
**** DB Management
**** Account Management
### *Hello Interface*
**** Hello Access
**** Remote Access
**** Door
**** Light
**** Batch
**** Recipe Start
**** Recipe End
**** Pumps
**** Acknowledge Alarms
**** Sensors
**** Hello Settings
### *Common*
**** Reports
**** Controls
**** System Management
# Item 4 - a listbox labeled Group Name
# Item 5 - a listbox labeled Password Expiration Period (days)
# Item 6 - a 2-D array of Booleans with no labels
## True will be bright green, and False will be dark green
## An empty row, for a nonexistent group, will be gray
# Item 7 - a scrollbar for scrolling up and down to see additional groups; it will not be labeled
# Item 8 - a Boolean button labeled Add Group
# Item 9 - a Boolean button labeled Delete Group
# Item 10 - a Boolean button labeled Save
# Item 11 - a Boolean button labeled Reset

*FRS2881.3.2* Configure Groups tab Functionality
# When a user makes any change in this menu, the Shell navigation buttons get grayed out so a user has to click the Save or Reset buttons.
# Item 1
## If unsaved changes are made in the Configure Groups tab, this tab will be grayed out.
## It will no longer be grayed out after clicking "Save" or "Reset".
# Item 4
## Only one item can be selected at all times
## The corresponding item in Item 5 will also be selected
## When an item is selected, the background will be blue and the text will be white
## When the Configure Groups tab is accessed, the first item in the list will be selected
## Empty rows cannot be selected
## A blank row will have gray controls for item 6, and items 4 and 5 will be blank
## Double clicking allows a user to modify the contents of item 4 for a particular group
## If the user clears the item and attempts to navigate away, it will be repopulated with the old value
## If the user enters the same name for item 4 for a group that already exists, it will be repopulated with the old value
# Item 5
## Only one item can be selected at all times
## The corresponding item in Item 4 will also be selected
## When an item is selected, the background will be blue and the text will be white
## When the Configure Groups tab is accessed, the first item in the list will be selected
## Empty rows cannot be selected
## A blank row will have gray controls for item 6, and items 4 and 5 will be blank
## Double clicking allows a user to modify the contents of item 5 for a particular group
## If the user clears the item and attempts to navigate away, it will be repopulated with 0
## Only integers greater than or equal to 0 will be entered
# Item 6 can be used to assign permissions to groups in this table
# Item 7
## Allows the user to scroll up and down to see groups beyond the 8 visible ones
## If there are less than 9 groups, the scrollbar shall not be usable.
## It will be applicable for Item 4, 5, and 6.
# Item 8
## If there is already a blank group and the user attempts to add another blank group, display the error message "Please edit the blank group first. Once the blank group is edited, you can add another group." 
## Clicking this will create a new group in the table, and select it.  The new group will be empty for item 4, be 0 for item 5, and item 6 will be all 'FALSE'
# Item 9
## Clicking this button will check the number of users assigned to the group, and the number of groups displayed.
### If it is the last group in the list, an alert VI (not a true popup) will be displayed:
**** The message will be "A group cannot be deleted if it is the only group remaining in the list"
**** The popup will have 1 button, "OK". 
**** When the "OK" button is clicked, the alert VI will close.
### If there are users assigned to the group, an alert VI (not a true popup) will be displayed:
**** The message will be "A group cannot be deleted if there are users assigned to it".
**** The popup will have 1 button, "OK". 
**** When the "OK" button is clicked, the alert VI will close.
### Otherwise, the selected group will be deleted from the table, and the top group in the list will be selected.
# Item 10
## Clicking this button will check that no groups have empty names (checking for duplicate names is done elsewhere).
### If any group names are blank, an alert VI (not a true popup) will be displayed:
**** The message will be "Groups with blank names cannot be saved"
**** The popup will have 1 button, "OK". 
**** When the "OK" button is clicked, the alert VI will close.
### If there are no user errors:
#### It will write the table to the PBSUsers.conf file 
#### A user event will be created, logging the contents of the table. 
#### The Shell navigation buttons that the user has permissions for will then no longer be grayed out.
#### Item 1 will no longer be grayed out.
# Item 11 will perform the following actions:
## Cancel all changes the user made
## Refresh the menu to show the current contents of the PBSUsers.conf file
## The Shell navigation buttons that the user has permissions for will then no longer be grayed out.
## Item 1 will no longer be grayed out.

*FRS2881.4:* PBSUsers.conf file
# The file will keep track of the following data per user:
** User Name
** Group
** Password
** Email Address
** Date Password was Saved
** Number of Failed Login Attempts
# The file will keep track of the following data per group:
** Group Name
** Password Expiration Period (Days)
# The file will keep a 2-D Permission table containing the following information:
** Permissions represented by columns.
** Groups represented by columns. 
** Individual permissions represented by entries in the table. 
"""
res = "\n".join(NumState(3).parse_text(lines))
print(res)
import clipboard
clipboard.copy(res)

*+Specification+*
*User Control Panel*
*FRS2881.0* Accessing
The user will click the "Account" button in the Shell, and the "Manage" button beneath it.
* *FRS2881.0.1:* If the user has "Account Management" permissions, they will be brought to the Configure Users tab of the User Control Panel, and the first name in the User List will be selected.
* *FRS2881.0.2:* If the user does not have "Account Management" permissions, the "Edit User" VI will open, for their own user account.

*FRS2881.1* Configure Users tab
Mockup
!Configure_Users.png!
Numbered mockup
!Configure_Users_Numbered.png!

*FRS2881.1.1* Configure Users tab Appearance
* Item 1 - Configure Users tab
** This will be a tab labeled "Configure Users"
** It will be the leftmost tab in the UI
* Item 2 - Configure Groups tab.
* Item 3 - User List
** It will be labeled User List
** It will have 2 columns
*** The first column will be labeled User Name
*** The second column will be labeled Group
* Item 4 - Edit User Boolean button
* I